In [1]:
# part 4 of ML project
from part4_fun import *
from check_fun import *


# main code block
if __name__ == '__main__':
    outfile = '/dev.p4.out'
    
    # loop over EN and FR languages
    for lang in ['EN', 'FR']:
        
        print ('============================',lang , '============================')
        
        # ======================================== training ========================================
        # reading tweets for particular language
        ptrain = data_from_file(lang + '/train') # unmodified
        train = mod_train2 (ptrain) # modified w/ start and stop states

        # getting sentiments/sentiment pairs and associated indices (w/ start and stop)
        sents = get_tags2(ptrain) 
        sent_pairs = get_tags2pairs(sents)
        print (sents)

        YY = get_counts2(train, sent_pairs) # dictionary of sentiment pairs and their counts
        diff_words = get_words(train)[0] # array of unique words 
        word_dict = get_words(train)[1] # dictionary of unique words and indices

        # emission and transmission parameter matrices
        emission_dict = get_emission2(train, 1) # dictionary with keys as (x, y) and values as emission probabilities
        em_mat = em_matrix(emission_dict, diff_words, sents) # emission matrix
        trans_mat2 = transition_params2(train, YY, sents, sent_pairs) # transition matrix
        trans_mat2 = mod_trans2(trans_mat2) # modifying transition matrix 
        # ========================================================================================== 
        non_zero_trans = transition_check(trans_mat2, 1e-15)
        
        
        # ==================================== validation set ====================================
        # A list of list of tuples of size 1. Each list in test is a tweet. 
        ptest = data_from_file(lang + '/dev.in')
        # test is a list of list. Each sublist is an array of words, 1 tweet
        ptest = [[word[0] for word in line] for line in ptest]
        test = mod_test2(ptest) # modified with start and stop words
        # ========================================================================================
        
        
        # ==================================== getting predictions ====================================
        # initializing list of optimal sentiment lists corresponding to each tweet 
        optimal_sentiments = []
        
        # loop that runs over all tweets for a given language to predict optimal sentiments
        for tweet in range(len(test)):
            
            # running Viterbi algorithm
            base_scores = np.ones([len(sents.keys()),1]) # initializing base case scores
            opt_ind_list = viterbi_algo2(em_mat, trans_mat2, word_dict, test[tweet], base_scores, base_scores, 2, []) 
            
            # generating list of optimal sentiments for a given sentence
            inv_sents = dict (zip(sents.values(), sents.keys())) # swapping keys and values
            opt_sents = [inv_sents[opt_ind_list[i]] for i in range(len(opt_ind_list))]
            
            # populating parent optimal sentiment list
            optimal_sentiments.append(opt_sents) 
            
            # printing iteration checks
            if (tweet % 100 == 0):
                print (tweet, ' out of ', len(test), ' tweets have been predicted.')
        
        predictions = [] # init list of predictions
        for tweet in range(len(optimal_sentiments)):
            predictions.append([(ptest[tweet][i], optimal_sentiments[tweet][i]) for i in range(len(optimal_sentiments[tweet]))])
        
        write_predictions(predictions, lang, outfile) # writing results to outfile
        # =============================================================================================
        
    
    print ('============================ Predictions Complete ============================')






============================ EN ============================
defaultdict(<class 'int'>, {'O': 2, 'B-INTJ': 3, 'B-PP': 4, 'B-NP': 5, 'I-NP': 6, 'B-VP': 7, 'B-PRT': 8, 'I-VP': 9, 'B-ADJP': 10, 'B-SBAR': 11, 'B-ADVP': 12, 'I-INTJ': 13, 'B-CONJP': 14, 'I-CONJP': 15, 'I-ADVP': 16, 'I-ADJP': 17, 'I-SBAR': 18, 'I-PP': 19, 'stop0': 20, 'stop1': 21, 'start0': 0, 'start1': 1})
There are 153 columns out of  484 with probabilities > 1e-15
0  out of  78  tweets have been predicted.
============================ FR ============================
defaultdict(<class 'int'>, {'O': 2, 'B-positive': 3, 'I-positive': 4, 'B-negative': 5, 'B-neutral': 6, 'I-negative': 7, 'I-neutral': 8, 'stop0': 9, 'stop1': 10, 'start0': 0, 'start1': 1})
There are 27 columns out of  121 with probabilities > 1e-15
0  out of  232  tweets have been predicted.
100  out of  232  tweets have been predicted.
200  out of  232  tweets have been predicted.
============================ Predictions Complete ============================
